In [14]:
import zipfile
import os

# Correct path to the ZIP file
zip_path = './PartB_DFU_Dataset_Infection.zip'

# Directory to extract to
extract_dir = './PartB_DFU_Dataset_Infection'

# Make sure the extraction directory exists
os.makedirs(extract_dir, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Extraction complete! Files are in: {extract_dir}")


Extraction complete! Files are in: ./PartB_DFU_Dataset_Infection


In [9]:
!pip install timm torchvision --quiet


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [16]:
import os, glob
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.utils as vutils
import timm
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Updated path to folder with 3500 images
image_dir = 'PartB_DFU_Dataset_Infection/PartB_DFU_Dataset/Infection/Aug-Negative'

# Image transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.image_paths = glob.glob(f"{root_dir}/**/*.jpg", recursive=True) + \
                           glob.glob(f"{root_dir}/**/*.png", recursive=True)
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, 0  # Dummy label

dataset = CustomImageDataset(image_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


In [17]:
class EfficientNetV2Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = timm.create_model('tf_efficientnetv2_m', pretrained=True, features_only=True)
        self.feature_dim = self.model.feature_info[-1]['num_chs']

    def forward(self, x):
        return self.model(x)[-1]  # Output: [B, C, H, W]

class LSTMGenerator(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, num_layers=2, img_size=224):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 3 * img_size * img_size)
        self.img_size = img_size

    def forward(self, x):
        # Flatten spatial dims
        x = x.view(x.size(0), -1, x.size(1))  # [B, HW, C]
        _, (hn, _) = self.lstm(x)
        out = self.fc(hn[-1])  # Take the last layer's hidden state
        out = out.view(-1, 3, self.img_size, self.img_size)
        return out



In [20]:
encoder = EfficientNetV2Encoder().to(device)
generator = LSTMGenerator(input_dim=encoder.feature_dim).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(generator.parameters(), lr=1e-4)

save_dir = "generated_img"
os.makedirs(save_dir, exist_ok=True)


Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.


In [21]:
epochs = 200

for epoch in range(epochs):
    for i, (imgs, _) in enumerate(tqdm(dataloader)):
        imgs = imgs.to(device)

        with torch.no_grad():
            features = encoder(imgs)

        gen_imgs = generator(features)
        loss = criterion(gen_imgs, imgs)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            vutils.save_image(gen_imgs[:4], f"{save_dir}/epoch{epoch}_batch{i}.png", normalize=True)

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


100%|██████████| 185/185 [00:38<00:00,  4.85it/s]


Epoch [1/200], Loss: 0.0439


100%|██████████| 185/185 [00:38<00:00,  4.77it/s]


Epoch [2/200], Loss: 0.0058


100%|██████████| 185/185 [00:39<00:00,  4.71it/s]


...

Epoch [198/200], Loss: 0.0109


100%|██████████| 185/185 [00:37<00:00,  4.97it/s]


Epoch [199/200], Loss: 0.0125


100%|██████████| 185/185 [00:40<00:00,  4.54it/s]

Epoch [200/200], Loss: 0.0389


In [5]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from torchmetrics.image.inception import InceptionScore
from torchmetrics.image.fid import FrechetInceptionDistance
from torchmetrics.image.psnr import PeakSignalNoiseRatio
from torchmetrics.image.ssim import StructuralSimilarityIndexMeasure

# Device setup (same as training)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset to load images from folder
class ImageFolderDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.image_paths = [
            os.path.join(folder_path, fname)
            for fname in os.listdir(folder_path)
            if fname.lower().endswith((".jpg", ".jpeg", ".png"))
        ]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, 0  # dummy label for dataloader

# Paths - adjust these based on your dataset
real_image_dir = "PartB_DFU_Dataset_Infection/PartB_DFU_Dataset/Infection/Aug-Negative"  # Your real images folder
generated_image_dir = "generated_img"  # Your saved generated images folder from training

# Image transform - resize and convert to tensor [0-1]
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# DataLoaders
real_loader = DataLoader(ImageFolderDataset(real_image_dir, transform), batch_size=16, shuffle=False)
gen_loader = DataLoader(ImageFolderDataset(generated_image_dir, transform), batch_size=16, shuffle=False)

# Initialize metrics on device
is_metric = InceptionScore().to(device)
fid_metric = FrechetInceptionDistance(feature=2048).to(device)
psnr_metric = PeakSignalNoiseRatio().to(device)
ssim_metric = StructuralSimilarityIndexMeasure().to(device)

# Update metrics with real images for FID (need uint8 [0-255])
for imgs, _ in real_loader:
    imgs_uint8 = (imgs * 255).clamp(0, 255).to(torch.uint8).to(device)
    fid_metric.update(imgs_uint8, real=True)

# Update metrics with generated images for FID and IS (uint8 [0-255])
for imgs, _ in gen_loader:
    imgs_uint8 = (imgs * 255).clamp(0, 255).to(torch.uint8).to(device)
    fid_metric.update(imgs_uint8, real=False)
    is_metric.update(imgs_uint8)

# Update PSNR & SSIM (float32 [0-1]) with generated images (compared to themselves here)
for imgs, _ in gen_loader:
    imgs = imgs.to(device)
    psnr_metric.update(imgs, imgs)
    ssim_metric.update(imgs, imgs)

# Compute final metric values
inception_score = is_metric.compute()[0].item()
fid_score = fid_metric.compute().item()
psnr_score = psnr_metric.compute().item()
ssim_score = ssim_metric.compute().item()

# Print results
print(f"Inception Score (IS): {inception_score:.2f}")
print(f"FID: {fid_score:.2f}")
print(f"PSNR: {psnr_score:.2f}")
print(f"SSIM: {ssim_score:.2f}")


Inception Score (IS): 1.55
FID: 397.83
PSNR: inf
SSIM: 1.00


In [23]:
!pip install torchmetrics --quiet



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [25]:
!pip install scikit-learn --quiet




[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [26]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix
)


In [28]:
!pip install torch-fidelity --quiet



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
